In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


batch_size = 256
learning_rate = 2e-4
num_epoch = 10

mnist_train = dset.MNIST("./", train=True, transform = transforms.ToTensor(),
                         target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform = transforms.ToTensor(),
                         target_transform=None, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                    shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                    shuffle=False, num_workers=2, drop_last=True)


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 5),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 3 * 3, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)

        return out


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_arr = []
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y = label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()

        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

    correct = 0
    total = 0

    with torch.no_grad():
        for image, label in test_loader:
            x = image.to(device)
            y = label.to(device)

            output = model.forward(x)
            _, output_index = torch.max(output, 1)

            total += label.size(0)
            correct += (output_index == y).sum().float()

        print("Accuracy of Test Data: {}".format(100 * correct / total))    

tensor(2.3445, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 98.11698913574219
tensor(0.1177, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 98.71794891357422
tensor(0.0649, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 99.00841522216797
tensor(0.0276, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 99.14863586425781
tensor(0.0189, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 99.21875
tensor(0.0174, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 99.21875
tensor(0.0139, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 99.18870544433594
tensor(0.0125, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 99.14863586425781
tensor(0.0082, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 99.19871520996094
tensor(0.0083, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 99.1786880493164
